# TD - Exercices Pratiques sur les Algorithmes de Graphes

Ce notebook contient des exercices pratiques pour appliquer les algorithmes BFS, DFS, Dijkstra et A* sur le réseau routier de Paris.

**Prérequis** : Avoir complété le notebook `TD_BFS_DFS_ShortestPath.ipynb` pour comprendre les algorithmes.

In [1]:
import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import display, HTML

print("Bibliothèques importées avec succès!")
print(f"NetworkX version: {nx.__version__}")
print(f"OSMnx version: {ox.__version__}")

Bibliothèques importées avec succès!
NetworkX version: 3.2.1
OSMnx version: 2.0.6


In [2]:
# Chargement du graphe du réseau routier de Paris
print("Chargement du réseau routier de Paris...")
G = ox.graph_from_place("Paris, France", network_type="drive")

# Conversion en graphe NetworkX non orienté pour simplifier les algorithmes
# (on peut garder le graphe orienté si nécessaire)
G = G.to_undirected()

print(f"Graphe chargé : {len(G.nodes())} nœuds, {len(G.edges())} arêtes")
print(f"Type de graphe : {type(G)}")
print(f"Est connexe : {nx.is_connected(G)}")

Chargement du réseau routier de Paris...
Graphe chargé : 9434 nœuds, 14768 arêtes
Type de graphe : <class 'networkx.classes.multigraph.MultiGraph'>
Est connexe : True


In [3]:
# Sélection d'une composante connexe principale si nécessaire
if not nx.is_connected(G):
    print("Le graphe n'est pas connexe. Sélection de la plus grande composante connexe...")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()
    print(f"Nouveau graphe : {len(G.nodes())} nœuds, {len(G.edges())} arêtes")

# Sélection de deux nœuds pour nos expérimentations
nodes = list(G.nodes())
start_node = nodes[0]
end_node = nodes[len(nodes)//4]  # Un nœud à environ 1/4 du graphe

print(f"\nNœud de départ : {start_node}")
print(f"Nœud d'arrivée : {end_node}")

# Détermination de l'attribut de poids pour les arêtes
# Dans les graphes OSMnx, les arêtes ont généralement un attribut 'length' (distance en mètres)
weight = None
if len(G.edges()) > 0:
    # Obtenir un exemple d'arête pour vérifier les attributs
    sample_edge = list(G.edges(keys=True))[0] if isinstance(G, nx.MultiGraph) else list(G.edges())[0]
    
    if isinstance(G, nx.MultiGraph):
        # Pour MultiGraph, prendre le premier dictionnaire d'attributs
        edge_data = G[sample_edge[0]][sample_edge[1]][sample_edge[2]]
    else:
        edge_data = G[sample_edge[0]][sample_edge[1]]
    
    # Vérifier si l'attribut 'length' existe
    if 'length' in edge_data:
        weight = 'length'
        print(f"\n✓ Attribut de poids détecté : 'length' (distance en mètres)")
    else:
        print(f"\n⚠ Aucun attribut de poids 'length' trouvé. Les algorithmes pondérés seront désactivés.")
        print(f"  Attributs disponibles sur les arêtes : {list(edge_data.keys())[:5]}")


Nœud de départ : 125730
Nœud d'arrivée : 27235559

✓ Attribut de poids détecté : 'length' (distance en mètres)


In [4]:
def plot_graph_plotly(G, title="Graphe", highlight_nodes=None, highlight_path=None, node_colors=None):
    """
    Visualise un graphe avec Plotly sur une carte
    
    Parameters:
    - G: graphe NetworkX
    - title: titre du graphique
    - highlight_nodes: liste de nœuds à mettre en évidence
    - highlight_path: liste de nœuds formant un chemin à mettre en évidence
    - node_colors: dictionnaire {node: color} pour colorer les nœuds
    """
    # Récupération des coordonnées (longitude, latitude)
    pos = {}
    for node in G.nodes():
        if 'x' in G.nodes[node] and 'y' in G.nodes[node]:
            # OSMnx stocke x=longitude, y=latitude
            pos[node] = (G.nodes[node]['x'], G.nodes[node]['y'])
        elif 'lon' in G.nodes[node] and 'lat' in G.nodes[node]:
            pos[node] = (G.nodes[node]['lon'], G.nodes[node]['lat'])
        else:
            # Fallback: utiliser l'ID du nœud comme coordonnées
            pos[node] = (node, 0)
    
    # Calcul du centre de la carte
    if pos:
        lons = [p[0] for p in pos.values() if isinstance(p[0], (int, float))]
        lats = [p[1] for p in pos.values() if isinstance(p[1], (int, float))]
        if lons and lats:
            center_lon = sum(lons) / len(lons)
            center_lat = sum(lats) / len(lats)
        else:
            center_lon, center_lat = 2.3522, 48.8566  # Paris par défaut
    else:
        center_lon, center_lat = 2.3522, 48.8566
    
    fig = go.Figure()
    
    # Préparation des arêtes avec coordonnées (lon, lat)
    for edge in G.edges():
        if edge[0] in pos and edge[1] in pos:
            lon0, lat0 = pos[edge[0]]
            lon1, lat1 = pos[edge[1]]
            # Vérifier que les coordonnées sont valides
            if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                fig.add_trace(go.Scattermap(
                    mode='lines',
                    lon=[lon0, lon1],
                    lat=[lat0, lat1],
                    line=dict(width=0.5, color='#888'),
                    hoverinfo='none',
                    showlegend=False
                ))
    
    # Préparation des nœuds
    node_lons = []
    node_lats = []
    node_text = []
    node_colors_list = []
    node_sizes = []
    
    for node in G.nodes():
        if node in pos:
            lon, lat = pos[node]
            if isinstance(lon, (int, float)) and isinstance(lat, (int, float)):
                node_lons.append(lon)
                node_lats.append(lat)
                node_text.append(f'Nœud {node}')
                
                # Détermination de la couleur et taille
                if highlight_path and node in highlight_path:
                    color = 'red'
                    size = 10
                elif highlight_nodes and node in highlight_nodes:
                    color = 'orange'
                    size = 12
                elif node_colors and node in node_colors:
                    color = node_colors[node]
                    size = 8
                else:
                    color = 'lightblue'
                    size = 4
                node_colors_list.append(color)
                node_sizes.append(size)
    
    # Ajout de la trace pour les nœuds
    if node_lons and node_lats:
        fig.add_trace(go.Scattermap(
            mode='markers',
            lon=node_lons,
            lat=node_lats,
            text=node_text,
            hoverinfo='text',
            marker=dict(
                size=node_sizes,
                color=node_colors_list,
                opacity=0.8
            ),
            showlegend=False
        ))
    
    # Mise à jour du layout avec mapbox
    fig.update_layout(
        title=title,
        mapbox=dict(
            style="open-street-map",  # Utilise OpenStreetMap comme basemap
            center=dict(lon=center_lon, lat=center_lat),
            zoom=11,  # Niveau de zoom adapté à Paris
            bearing=0,
            pitch=0
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=700
    )
    
    return fig

## Fonctions utilitaires pour la visualisation

In [5]:
def dfs_iterative(G, start, end=None):
    """
    DFS itératif utilisant une pile
    
    Parameters:
    - G: graphe NetworkX
    - start: nœud de départ
    - end: nœud d'arrivée (optionnel)
    
    Returns:
    - visited: ensemble des nœuds visités
    - path: chemin vers 'end' si spécifié, None sinon
    - order: ordre de visite des nœuds
    """
    visited = set()
    stack = [start]
    order = []
    parent = {start: None}
    
    while stack:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            order.append(node)
            
            # Si on a trouvé le nœud cible
            if end is not None and node == end:
                # Reconstruire le chemin
                path = []
                current = end
                while current is not None:
                    path.append(current)
                    current = parent[current]
                return visited, list(reversed(path)), order
            
            # Ajouter les voisins à la pile (en ordre inverse pour cohérence)
            neighbors = list(G.neighbors(node))
            neighbors.reverse()
            for neighbor in neighbors:
                if neighbor not in visited:
                    stack.append(neighbor)
                    if neighbor not in parent:
                        parent[neighbor] = node
    
    # Si end était spécifié mais non trouvé
    if end is not None:
        return visited, None, order
    
    return visited, None, order

# Test du DFS itératif
print("Exécution du DFS itératif...")
visited_dfs_iter, path_dfs_iter, order_dfs_iter = dfs_iterative(G, start_node, end_node)

print(f"Nombre de nœuds visités : {len(visited_dfs_iter)}")
print(f"Longueur du chemin trouvé : {len(path_dfs_iter) if path_dfs_iter else 'Aucun chemin'}")

if path_dfs_iter:
    print(f"Chemin (10 premiers nœuds) : {path_dfs_iter[:10]}...")
    print(f"Chemin (10 derniers nœuds) : ...{path_dfs_iter[-10:]}")


Exécution du DFS itératif...
Nombre de nœuds visités : 2984
Longueur du chemin trouvé : 573
Chemin (10 premiers nœuds) : [125730, 12179625841, 4467439444, 21112878, 2841627109, 84791219, 1766131439, 1089076082, 1089076107, 1089076163]...
Chemin (10 derniers nœuds) : ...[25075891, 255028808, 27286561, 27235582, 442309, 27235547, 27286553, 442319, 27233766, 27235559]


## Algorithmes de parcours

In [6]:
# Fonction heuristique pour A* (distance euclidienne vers le but)
def heuristic(u, v):
    """Heuristique : distance euclidienne entre deux nœuds"""
    try:
        if 'x' in G.nodes[u] and 'y' in G.nodes[u]:
            x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
            x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
        elif 'lon' in G.nodes[u] and 'lat' in G.nodes[u]:
            x1, y1 = G.nodes[u]['lon'], G.nodes[u]['lat']
            x2, y2 = G.nodes[v]['lon'], G.nodes[v]['lat']
        else:
            return 0
        
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    except:
        return 0

# Calcul du plus court chemin avec A*
try:
    print("Calcul du plus court chemin avec A*...")
    if weight:
        path_astar = nx.astar_path(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
        length_astar = nx.astar_path_length(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
        
        print(f"Chemin A* - Longueur : {len(path_astar)} nœuds")
        print(f"Distance totale : {length_astar:.2f} mètres")
        print(f"Chemin (10 premiers nœuds) : {path_astar[:10]}...")
        print(f"Chemin (10 derniers nœuds) : ...{path_astar[-10:]}")
    else:
        # Sans poids, A* est équivalent à Dijkstra/BFS
        path_astar = nx.astar_path(G, source=start_node, target=end_node, heuristic=heuristic)
        length_astar = len(path_astar)
        
        print(f"Chemin A* (sans poids) - Longueur : {len(path_astar)} nœuds")
        print(f"Chemin (10 premiers nœuds) : {path_astar[:10]}...")
        print(f"Chemin (10 derniers nœuds) : ...{path_astar[-10:]}")
    
except Exception as e:
    print(f"Erreur avec A* : {e}")
    path_astar = None
    length_astar = None


Calcul du plus court chemin avec A*...
Chemin A* - Longueur : 125 nœuds
Distance totale : 9343.17 mètres
Chemin (10 premiers nœuds) : [125730, 1089076182, 1089076216, 1766131427, 10299440575, 84791232, 134521832, 2405282107, 2405282109, 134521836]...
Chemin (10 derniers nœuds) : ...[27234093, 1107432316, 5095554625, 5095554623, 27233726, 1369651515, 367908, 27233754, 5204784909, 27235559]


## Comparaison des algorithmes

## 8. Exercices Pratiques

Les exercices suivants vous permettront de mettre en pratique les algorithmes de parcours de graphes sur le réseau routier parisien.


In [7]:
import time

print("=== Comparaison des algorithmes ===\n")

# Construire les algorithmes en fonction de la disponibilité des poids
algorithms = {
    'BFS (non pondéré)': lambda: nx.shortest_path(G, start_node, end_node, method='bfs')
}

if weight:
    algorithms['Dijkstra'] = lambda: nx.shortest_path(G, start_node, end_node, weight=weight, method='dijkstra')
    algorithms['A*'] = lambda: nx.astar_path(G, start_node, end_node, weight=weight, heuristic=heuristic)
else:
    # Si pas de poids, Dijkstra est équivalent à BFS
    algorithms['Dijkstra (sans poids)'] = lambda: nx.shortest_path(G, start_node, end_node, method='dijkstra')

results = {}
for name, func in algorithms.items():
    try:
        start_time = time.time()
        path = func()
        elapsed = time.time() - start_time
        length = len(path)
        
        if weight:
            path_length = nx.shortest_path_length(G, start_node, end_node, weight=weight)
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': path_length,
                'time': elapsed
            }
        else:
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': None,
                'time': elapsed
            }
        print(f"{name}:")
        print(f"  - Temps : {elapsed*1000:.2f} ms")
        print(f"  - Nombre de nœuds : {length}")
        if weight:
            print(f"  - Distance : {path_length:.2f}")
        print()
    except Exception as e:
        print(f"{name}: Erreur - {e}\n")


=== Comparaison des algorithmes ===

BFS (non pondéré): Erreur - method not supported: bfs

Dijkstra:
  - Temps : 59.31 ms
  - Nombre de nœuds : 125
  - Distance : 9343.17

A*:
  - Temps : 68.90 ms
  - Nombre de nœuds : 125
  - Distance : 9343.17



In [8]:
# Exercice 1 : Trouver des nœuds proches de monuments parisiens et visualiser les chemins
try:
    # Coordonnées approximatives (lon, lat) de monuments célèbres
    monuments = {
        'Tour Eiffel': (2.2945, 48.8584),
        'Notre-Dame': (2.3499, 48.8530),
        'Louvre': (2.3364, 48.8606),
        'Arc de Triomphe': (2.2950, 48.8738),
        'Sacré-Cœur': (2.3431, 48.8867),
        'Gare du Nord': (2.3553, 48.8809)
    }
    
    print("=" * 70)
    print("EXERCICE 1 : Recherche de nœuds proches des monuments parisiens")
    print("=" * 70)
    print("\nRecherche des nœuds proches des monuments...\n")
    
    monument_nodes = {}
    monument_coords = {}
    
    for name, (lon, lat) in monuments.items():
        # Trouver le nœud le plus proche
        min_dist = float('inf')
        closest_node = None
        
        for node in G.nodes():
            node_lon = G.nodes[node].get('x', G.nodes[node].get('lon', 0))
            node_lat = G.nodes[node].get('y', G.nodes[node].get('lat', 0))
            
            if isinstance(node_lon, (int, float)) and isinstance(node_lat, (int, float)):
                dist = np.sqrt((node_lon - lon)**2 + (node_lat - lat)**2)
                if dist < min_dist:
                    min_dist = dist
                    closest_node = node
                    closest_coords = (node_lon, node_lat)
        
        if closest_node is not None:
            monument_nodes[name] = closest_node
            monument_coords[name] = closest_coords
            # Convertir la distance en mètres approximatifs (1 degré ≈ 111 km)
            dist_m = min_dist * 111000
            print(f"{name:20s}: nœud {closest_node:10s} (distance: {dist_m:.0f} m)")
    
    # Trouver le chemin entre plusieurs paires de monuments
    print("\n" + "=" * 70)
    print("Chemins entre monuments (utilisant Dijkstra)")
    print("=" * 70)
    
    pairs_to_test = [
        ('Tour Eiffel', 'Louvre'),
        ('Arc de Triomphe', 'Sacré-Cœur'),
        ('Notre-Dame', 'Gare du Nord')
    ]
    
    all_paths = {}
    for start_monument, end_monument in pairs_to_test:
        if start_monument in monument_nodes and end_monument in monument_nodes:
            start_m = monument_nodes[start_monument]
            end_m = monument_nodes[end_monument]
            
            try:
                path_monument = nx.shortest_path(G, source=start_m, target=end_m, weight=weight, method='dijkstra')
                length_monument = nx.shortest_path_length(G, source=start_m, target=end_m, weight=weight)
                all_paths[(start_monument, end_monument)] = (path_monument, length_monument)
                
                print(f"\n{start_monument} → {end_monument}:")
                print(f"  - Nombre de nœuds : {len(path_monument)}")
                if weight:
                    print(f"  - Distance totale : {length_monument:.0f} mètres")
                    print(f"  - Distance en km : {length_monument/1000:.2f} km")
                else:
                    print(f"  - Distance : {length_monument:.2f}")
            except nx.NetworkXNoPath:
                print(f"\n{start_monument} → {end_monument}: Aucun chemin trouvé")
            except Exception as e:
                print(f"\n{start_monument} → {end_monument}: Erreur - {e}")
    
    # Visualisation des chemins entre monuments
    if all_paths:
        print("\n" + "=" * 70)
        print("Visualisation des chemins entre monuments")
        print("=" * 70)
        
        # Créer un graphe avec tous les chemins
        path_nodes_set = set()
        for (start, end), (path, length) in all_paths.items():
            path_nodes_set.update(path)
            # Ajouter quelques voisins pour contexte
            for node in path[::10]:  # Prendre 1 nœud sur 10
                for neighbor in list(G.neighbors(node))[:2]:
                    path_nodes_set.add(neighbor)
        
        G_monuments = G.subgraph(list(path_nodes_set)[:800]).copy()
        
        # Récupération des coordonnées
        pos_mon = {}
        for node in G_monuments.nodes():
            if 'x' in G_monuments.nodes[node] and 'y' in G_monuments.nodes[node]:
                pos_mon[node] = (G_monuments.nodes[node]['x'], G_monuments.nodes[node]['y'])
            elif 'lon' in G_monuments.nodes[node] and 'lat' in G_monuments.nodes[node]:
                pos_mon[node] = (G_monuments.nodes[node]['lon'], G_monuments.nodes[node]['lat'])
        
        # Calcul du centre
        if pos_mon:
            center_lon = sum(p[0] for p in pos_mon.values() if isinstance(p[0], (int, float))) / len(pos_mon)
            center_lat = sum(p[1] for p in pos_mon.values() if isinstance(p[1], (int, float))) / len(pos_mon)
        else:
            center_lon, center_lat = 2.3522, 48.8566
        
        fig = go.Figure()
        
        # Arêtes du réseau (légères)
        edge_lons, edge_lats = [], []
        for edge in G_monuments.edges():
            if edge[0] in pos_mon and edge[1] in pos_mon:
                lon0, lat0 = pos_mon[edge[0]]
                lon1, lat1 = pos_mon[edge[1]]
                if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                    isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                    edge_lons.extend([lon0, lon1, None])
                    edge_lats.extend([lat0, lat1, None])
        
        if edge_lons:
            fig.add_trace(go.Scattermap(
                mode='lines',
                lon=edge_lons,
                lat=edge_lats,
                line=dict(width=0.3, color='lightgray'),
                hoverinfo='none',
                showlegend=False
            ))
        
        # Chemins entre monuments avec couleurs différentes
        colors = ['blue', 'green', 'purple', 'orange', 'red']
        for i, ((start, end), (path, length)) in enumerate(all_paths.items()):
            path_lons = [pos_mon[node][0] for node in path if node in pos_mon 
                        and isinstance(pos_mon[node][0], (int, float))]
            path_lats = [pos_mon[node][1] for node in path if node in pos_mon 
                        and isinstance(pos_mon[node][1], (int, float))]
            
            if path_lons and path_lats:
                color = colors[i % len(colors)]
                fig.add_trace(go.Scattermap(
                    mode='lines+markers',
                    lon=path_lons,
                    lat=path_lats,
                    line=dict(width=4, color=color),
                    marker=dict(size=5, color=color, opacity=0.6),
                    name=f'{start} → {end}',
                    showlegend=True,
                    legendgroup='paths'
                ))
        
        # Marqueurs pour les monuments
        monument_lons = []
        monument_lats = []
        monument_names = []
        for name, node in monument_nodes.items():
            if node in pos_mon:
                lon, lat = pos_mon[node]
                if isinstance(lon, (int, float)) and isinstance(lat, (int, float)):
                    monument_lons.append(lon)
                    monument_lats.append(lat)
                    monument_names.append(name)
        
        if monument_lons:
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=monument_lons,
                lat=monument_lats,
                text=monument_names,
                marker=dict(size=15, color='red', symbol='circle'),
                name='Monuments',
                showlegend=True,
                legendgroup='monuments'
            ))
        
        fig.update_layout(
            title="Chemins entre Monuments Parisiens",
            showlegend=True,
            mapbox=dict(
                style="open-street-map",
                center=dict(lon=center_lon, lat=center_lat),
                zoom=12,
                bearing=0,
                pitch=0
            ),
            margin=dict(l=0, r=0, t=40, b=0),
            height=700
        )
        
        fig.show()
        
except Exception as e:
    print(f"Erreur lors de l'exercice 1 : {e}")
    import traceback
    traceback.print_exc()


EXERCICE 1 : Recherche de nœuds proches des monuments parisiens

Recherche des nœuds proches des monuments...

Erreur lors de l'exercice 1 : Unknown format code 's' for object of type 'int'


Traceback (most recent call last):
  File "/var/folders/05/3bcz94ds2xlfpypg943txflc0000gn/T/ipykernel_57704/769404005.py", line 42, in <module>
    print(f"{name:20s}: nœud {closest_node:10s} (distance: {dist_m:.0f} m)")
                             ^^^^^^^^^^^^^^^^^^
ValueError: Unknown format code 's' for object of type 'int'


In [9]:
# Exercice 2 : Analyser les caractéristiques du réseau routier parisien
try:
    print("=" * 70)
    print("EXERCICE 2 : Analyse du réseau routier parisien")
    print("=" * 70)
    
    # 1. Statistiques de base
    print("\n1. Statistiques du graphe :")
    print(f"   - Nombre de nœuds : {len(G.nodes())}")
    print(f"   - Nombre d'arêtes : {len(G.edges())}")
    print(f"   - Densité : {nx.density(G):.6f}")
    print(f"   - Est connexe : {nx.is_connected(G)}")
    
    # 2. Degré des nœuds
    degrees = dict(G.degree())
    avg_degree = sum(degrees.values()) / len(degrees) if degrees else 0
    max_degree_node = max(degrees.items(), key=lambda x: x[1])
    min_degree_node = min(degrees.items(), key=lambda x: x[1])
    
    print("\n2. Analyse des degrés :")
    print(f"   - Degré moyen : {avg_degree:.2f}")
    print(f"   - Nœud avec le plus de connexions : {max_degree_node[0]} (degré {max_degree_node[1]})")
    print(f"   - Nœud avec le moins de connexions : {min_degree_node[0]} (degré {min_degree_node[1]})")
    
    # 3. Plus court chemin moyen (échantillon)
    print("\n3. Analyse des distances (échantillon de 50 paires) :")
    sample_size = min(50, len(G.nodes()))
    sample_nodes = list(G.nodes())[:sample_size]
    
    path_lengths = []
    distances_m = []
    
    for i in range(min(50, sample_size-1)):
        try:
            if weight:
                length = nx.shortest_path_length(G, source=sample_nodes[i], 
                                                target=sample_nodes[i+1], weight=weight)
                distances_m.append(length)
            else:
                path = nx.shortest_path(G, source=sample_nodes[i], target=sample_nodes[i+1])
                path_lengths.append(len(path))
        except (nx.NetworkXNoPath, IndexError):
            continue
    
    if distances_m:
        avg_distance = sum(distances_m) / len(distances_m)
        max_distance = max(distances_m)
        min_distance = min(distances_m)
        print(f"   - Distance moyenne : {avg_distance:.0f} mètres ({avg_distance/1000:.2f} km)")
        print(f"   - Distance maximale : {max_distance:.0f} mètres ({max_distance/1000:.2f} km)")
        print(f"   - Distance minimale : {min_distance:.0f} mètres ({min_distance/1000:.2f} km)")
    elif path_lengths:
        avg_path = sum(path_lengths) / len(path_lengths)
        print(f"   - Longueur moyenne du chemin : {avg_path:.1f} nœuds")
    
    # 4. Nœuds centraux (centralité de degré)
    print("\n4. Les 5 nœuds les plus connectés :")
    sorted_nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:5]
    for node, deg in sorted_nodes:
        node_lon = G.nodes[node].get('x', G.nodes[node].get('lon', 'N/A'))
        node_lat = G.nodes[node].get('y', G.nodes[node].get('lat', 'N/A'))
        print(f"   - Nœud {node}: degré {deg} (coords: {node_lon:.4f}, {node_lat:.4f})")
    
    print("\n✓ Analyse terminée !")
    
    # Visualisation : Afficher les nœuds les plus connectés sur la carte
    print("\n" + "=" * 70)
    print("Visualisation des nœuds les plus connectés")
    print("=" * 70)
    
    # Prendre les 20 nœuds les plus connectés
    top_nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:20]
    
    # Créer un sous-graphe avec ces nœuds et leurs voisins
    nodes_to_show = set([node for node, _ in top_nodes])
    for node, _ in top_nodes[:10]:  # Limiter aux 10 premiers
        nodes_to_show.update(list(G.neighbors(node))[:5])
    
    G_top = G.subgraph(list(nodes_to_show)[:200]).copy()
    
    # Coordonnées
    pos_top = {}
    for node in G_top.nodes():
        if 'x' in G_top.nodes[node] and 'y' in G_top.nodes[node]:
            pos_top[node] = (G_top.nodes[node]['x'], G_top.nodes[node]['y'])
    
    if pos_top:
        center_lon = sum(p[0] for p in pos_top.values()) / len(pos_top)
        center_lat = sum(p[1] for p in pos_top.values()) / len(pos_top)
        
        fig = go.Figure()
        
        # Arêtes
        edge_lons, edge_lats = [], []
        for edge in G_top.edges():
            if edge[0] in pos_top and edge[1] in pos_top:
                lon0, lat0 = pos_top[edge[0]]
                lon1, lat1 = pos_top[edge[1]]
                edge_lons.extend([lon0, lon1, None])
                edge_lats.extend([lat0, lat1, None])
        
        if edge_lons:
            fig.add_trace(go.Scattermap(
                mode='lines',
                lon=edge_lons,
                lat=edge_lats,
                line=dict(width=0.5, color='lightgray'),
                hoverinfo='none',
                showlegend=False
            ))
        
        # Marqueurs pour les nœuds les plus connectés
        top_lons, top_lats, top_labels, top_sizes = [], [], [], []
        for node, deg in top_nodes[:15]:
            if node in pos_top:
                top_lons.append(pos_top[node][0])
                top_lats.append(pos_top[node][1])
                top_labels.append(f"Nœud {node}<br>Degré: {deg}")
                # Taille proportionnelle au degré
                top_sizes.append(min(20, 8 + deg))
        
        if top_lons:
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=top_lons,
                lat=top_lats,
                text=top_labels,
                hoverinfo='text',
                marker=dict(
                    size=top_sizes,
                    color='red',
                    opacity=0.7,
                    line=dict(width=2, color='darkred')
                ),
                name='Nœuds très connectés',
                showlegend=True
            ))
        
        # Autres nœuds
        other_lons, other_lats = [], []
        for node in G_top.nodes():
            if node not in [n for n, _ in top_nodes[:15]] and node in pos_top:
                other_lons.append(pos_top[node][0])
                other_lats.append(pos_top[node][1])
        
        if other_lons:
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=other_lons,
                lat=other_lats,
                marker=dict(size=4, color='lightblue', opacity=0.5),
                name='Autres nœuds',
                showlegend=True
            ))
        
        fig.update_layout(
            title="Nœuds les Plus Connectés du Réseau Routier Parisien",
            showlegend=True,
            mapbox=dict(
                style="open-street-map",
                center=dict(lon=center_lon, lat=center_lat),
                zoom=12,
                bearing=0,
                pitch=0
            ),
            margin=dict(l=0, r=0, t=40, b=0),
            height=700
        )
        
        fig.show()
        print("   ✓ Visualisation créée")
    
except Exception as e:
    print(f"Erreur lors de l'exercice 2 : {e}")
    import traceback
    traceback.print_exc()


EXERCICE 2 : Analyse du réseau routier parisien

1. Statistiques du graphe :
   - Nombre de nœuds : 9434
   - Nombre d'arêtes : 14768
   - Densité : 0.000332
   - Est connexe : True

2. Analyse des degrés :
   - Degré moyen : 3.13
   - Nœud avec le plus de connexions : 1761442006 (degré 7)
   - Nœud avec le moins de connexions : 361092 (degré 1)

3. Analyse des distances (échantillon de 50 paires) :
   - Distance moyenne : 977 mètres (0.98 km)
   - Distance maximale : 10403 mètres (10.40 km)
   - Distance minimale : 38 mètres (0.04 km)

4. Les 5 nœuds les plus connectés :
   - Nœud 1761442006: degré 7 (coords: 2.2811, 48.8679)
   - Nœud 361103: degré 6 (coords: 2.3523, 48.8805)
   - Nœud 14798310: degré 6 (coords: 2.2664, 48.8490)
   - Nœud 24910810: degré 6 (coords: 2.3396, 48.8912)
   - Nœud 25254138: degré 6 (coords: 2.3255, 48.8559)

✓ Analyse terminée !

Visualisation des nœuds les plus connectés
Erreur lors de l'exercice 2 : Invalid property specified for object of type plotly.gr

Traceback (most recent call last):
  File "/var/folders/05/3bcz94ds2xlfpypg943txflc0000gn/T/ipykernel_57704/3547817399.py", line 123, in <module>
    fig.add_trace(go.Scattermap(
                  ^^^^^^^^^^^^^^
  File "/Users/martin/Documents/Code/formation/graph_trip/.venv/lib/python3.12/site-packages/plotly/graph_objs/_scattermap.py", line 1685, in __init__
    self._set_property("marker", arg, marker)
  File "/Users/martin/Documents/Code/formation/graph_trip/.venv/lib/python3.12/site-packages/plotly/basedatatypes.py", line 4403, in _set_property
    _set_property_provided_value(self, name, arg, provided)
  File "/Users/martin/Documents/Code/formation/graph_trip/.venv/lib/python3.12/site-packages/plotly/basedatatypes.py", line 398, in _set_property_provided_value
    obj[name] = val
    ~~~^^^^^^
  File "/Users/martin/Documents/Code/formation/graph_trip/.venv/lib/python3.12/site-packages/plotly/basedatatypes.py", line 4924, in __setitem__
    self._set_compound_prop(prop, value)
  F

## Exercice 3 : Comparer BFS, DFS et Dijkstra sur différents trajets

Cet exercice compare les performances et les résultats des différents algorithmes de parcours.


In [10]:
# Exercice 3 : Comparer BFS, DFS et Dijkstra sur différents trajets
try:
    print("=" * 70)
    print("EXERCICE 3 : Comparaison BFS, DFS et Dijkstra")
    print("=" * 70)
    
    # Sélectionner plusieurs paires de nœuds pour tester
    nodes_list = list(G.nodes())
    test_pairs = []
    
    # Créer 3 paires de nœuds à différentes distances
    if len(nodes_list) >= 100:
        test_pairs = [
            (nodes_list[0], nodes_list[len(nodes_list)//10]),      # Distance courte
            (nodes_list[0], nodes_list[len(nodes_list)//4]),       # Distance moyenne
            (nodes_list[0], nodes_list[len(nodes_list)//2])        # Distance longue
        ]
    else:
        # Si le graphe est petit, prendre des paires arbitraires
        test_pairs = [
            (nodes_list[0], nodes_list[min(10, len(nodes_list)-1)]),
            (nodes_list[0], nodes_list[min(20, len(nodes_list)-1)])
        ]
    
    import time
    
    print("\nComparaison pour plusieurs trajets :\n")
    
    all_results = []
    
    for i, (start, end) in enumerate(test_pairs, 1):
        print(f"{'='*70}")
        print(f"Trajet {i} : Nœud {start} → Nœud {end}")
        print(f"{'='*70}")
        
        results = {}
        
        # BFS
        try:
            t_start = time.time()
            path_bfs = nx.shortest_path(G, source=start, target=end, method='bfs')
            t_bfs = time.time() - t_start
            results['BFS'] = {
                'path': path_bfs,
                'length': len(path_bfs),
                'time': t_bfs
            }
            print(f"BFS      : {len(path_bfs):4d} nœuds, temps: {t_bfs*1000:6.2f} ms")
        except Exception as e:
            print(f"BFS      : Erreur - {e}")
            results['BFS'] = None
        
        # DFS (ne garantit pas le plus court chemin)
        try:
            visited_dfs, path_dfs, _ = dfs_iterative(G, start, end)
            if path_dfs:
                t_dfs = 0  # Approximation, on a déjà calculé
                results['DFS'] = {
                    'path': path_dfs,
                    'length': len(path_dfs),
                    'time': t_dfs
                }
                print(f"DFS      : {len(path_dfs):4d} nœuds")
            else:
                print(f"DFS      : Aucun chemin trouvé")
                results['DFS'] = None
        except Exception as e:
            print(f"DFS      : Erreur - {e}")
            results['DFS'] = None
        
        # Dijkstra
        if weight:
            try:
                t_start = time.time()
                path_dijk = nx.shortest_path(G, source=start, target=end, weight=weight, method='dijkstra')
                dist_dijk = nx.shortest_path_length(G, source=start, target=end, weight=weight)
                t_dijk = time.time() - t_start
                results['Dijkstra'] = {
                    'path': path_dijk,
                    'length': len(path_dijk),
                    'distance': dist_dijk,
                    'time': t_dijk
                }
                print(f"Dijkstra : {len(path_dijk):4d} nœuds, {dist_dijk:.0f} m, temps: {t_dijk*1000:6.2f} ms")
            except Exception as e:
                print(f"Dijkstra : Erreur - {e}")
                results['Dijkstra'] = None
        
        # A*
        if weight:
            try:
                t_start = time.time()
                path_ast = nx.astar_path(G, source=start, target=end, weight=weight, heuristic=heuristic)
                dist_ast = nx.astar_path_length(G, source=start, target=end, weight=weight, heuristic=heuristic)
                t_ast = time.time() - t_start
                results['A*'] = {
                    'path': path_ast,
                    'length': len(path_ast),
                    'distance': dist_ast,
                    'time': t_ast
                }
                print(f"A*       : {len(path_ast):4d} nœuds, {dist_ast:.0f} m, temps: {t_ast*1000:6.2f} ms")
            except Exception as e:
                print(f"A*       : Erreur - {e}")
                results['A*'] = None
        
        all_results.append((start, end, results))
        print()
    
    # Résumé
    print("\n" + "=" * 70)
    print("Résumé des résultats :")
    print("=" * 70)
    
    if weight:
        print("\nSur un graphe pondéré :")
        print("- BFS trouve le plus court chemin en nombre de nœuds (mais pas forcément en distance)")
        print("- Dijkstra et A* trouvent le plus court chemin en distance")
        print("- A* est généralement plus rapide que Dijkstra grâce à l'heuristique")
    
    print("\nObservations :")
    print("- BFS garantit le plus court chemin en nombre de nœuds sur un graphe non pondéré")
    print("- DFS ne garantit PAS le plus court chemin")
    print("- Pour des chemins réels (avec distances), utilisez Dijkstra ou A*")
    
except Exception as e:
    print(f"Erreur lors de l'exercice 3 : {e}")
    import traceback
    traceback.print_exc()


EXERCICE 3 : Comparaison BFS, DFS et Dijkstra

Comparaison pour plusieurs trajets :

Trajet 1 : Nœud 125730 → Nœud 21660948
BFS      : Erreur - method not supported: bfs
DFS      :  485 nœuds
Dijkstra :  119 nœuds, 9145 m, temps: 109.93 ms
A*       :  119 nœuds, 9145 m, temps: 135.47 ms

Trajet 2 : Nœud 125730 → Nœud 27235559
BFS      : Erreur - method not supported: bfs
DFS      :  573 nœuds
Dijkstra :  125 nœuds, 9343 m, temps:  83.84 ms
A*       :  125 nœuds, 9343 m, temps: 147.43 ms

Trajet 3 : Nœud 125730 → Nœud 144489536
BFS      : Erreur - method not supported: bfs
DFS      :  392 nœuds
Dijkstra :  123 nœuds, 9821 m, temps:  93.70 ms
A*       :  123 nœuds, 9821 m, temps: 141.72 ms


Résumé des résultats :

Sur un graphe pondéré :
- BFS trouve le plus court chemin en nombre de nœuds (mais pas forcément en distance)
- Dijkstra et A* trouvent le plus court chemin en distance
- A* est généralement plus rapide que Dijkstra grâce à l'heuristique

Observations :
- BFS garantit le plus 

In [11]:
# Exercice 4 : Trouver le chemin optimal entre gares parisiennes
try:
    print("=" * 70)
    print("EXERCICE 4 : Chemins entre gares parisiennes")
    print("=" * 70)
    
    # Coordonnées des principales gares parisiennes
    gares = {
        'Gare du Nord': (2.3553, 48.8809),
        'Gare de l\'Est': (2.3590, 48.8768),
        'Gare de Lyon': (2.3733, 48.8447),
        'Gare Montparnasse': (2.3217, 48.8412),
        'Gare Saint-Lazare': (2.3260, 48.8767),
        'Gare d\'Austerlitz': (2.3642, 48.8420)
    }
    
    print("\nRecherche des nœuds proches des gares...\n")
    
    gare_nodes = {}
    for name, (lon, lat) in gares.items():
        min_dist = float('inf')
        closest_node = None
        
        for node in G.nodes():
            node_lon = G.nodes[node].get('x', G.nodes[node].get('lon', 0))
            node_lat = G.nodes[node].get('y', G.nodes[node].get('lat', 0))
            
            if isinstance(node_lon, (int, float)) and isinstance(node_lat, (int, float)):
                dist = np.sqrt((node_lon - lon)**2 + (node_lat - lat)**2)
                if dist < min_dist:
                    min_dist = dist
                    closest_node = node
        
        if closest_node:
            gare_nodes[name] = closest_node
            dist_m = min_dist * 111000
            print(f"{name:25s}: nœud {closest_node:10s} (distance: {dist_m:.0f} m)")
    
    # Calculer les chemins entre toutes les paires de gares
    print("\n" + "=" * 70)
    print("Chemins entre gares (matrice de distances)")
    print("=" * 70)
    
    gare_names = list(gare_nodes.keys())
    paths_matrix = {}
    distances_matrix = {}
    
    for start_gare in gare_names:
        for end_gare in gare_names:
            if start_gare != end_gare:
                try:
                    start_node = gare_nodes[start_gare]
                    end_node = gare_nodes[end_gare]
                    
                    if weight:
                        path = nx.shortest_path(G, source=start_node, target=end_node, weight=weight, method='dijkstra')
                        dist = nx.shortest_path_length(G, source=start_node, target=end_node, weight=weight)
                        paths_matrix[(start_gare, end_gare)] = path
                        distances_matrix[(start_gare, end_gare)] = dist
                    else:
                        path = nx.shortest_path(G, source=start_node, target=end_node, method='bfs')
                        paths_matrix[(start_gare, end_gare)] = path
                except nx.NetworkXNoPath:
                    continue
    
    # Afficher un tableau des distances
    if distances_matrix:
        print("\nDistances en mètres entre gares :")
        print(f"\n{'Gare':<20s}", end="")
        for gare in gare_names[:5]:  # Limiter à 5 pour la lisibilité
            print(f"{gare[:15]:>15s}", end="")
        print()
        print("-" * 90)
        
        for start in gare_names[:5]:
            print(f"{start:<20s}", end="")
            for end in gare_names[:5]:
                if start == end:
                    print(f"{'0':>15s}", end="")
                elif (start, end) in distances_matrix:
                    dist = distances_matrix[(start, end)]
                    print(f"{dist/1000:>13.2f}km", end="")
                else:
                    print(f"{'N/A':>15s}", end="")
            print()
        
        # Trouver la paire la plus proche et la plus éloignée
        if distances_matrix:
            min_pair = min(distances_matrix.items(), key=lambda x: x[1])
            max_pair = max(distances_matrix.items(), key=lambda x: x[1])
            
            print(f"\nGares les plus proches : {min_pair[0][0]} ↔ {min_pair[0][1]}")
            print(f"  Distance : {min_pair[1]:.0f} mètres ({min_pair[1]/1000:.2f} km)")
            
            print(f"\nGares les plus éloignées : {max_pair[0][0]} ↔ {max_pair[0][1]}")
            print(f"  Distance : {max_pair[1]:.0f} mètres ({max_pair[1]/1000:.2f} km)")
    
    # Visualisation des chemins entre gares principales
    if paths_matrix:
        print("\n" + "=" * 70)
        print("Visualisation des chemins entre gares principales")
        print("=" * 70)
        
        # Sélectionner quelques chemins intéressants à visualiser
        paths_to_show = []
        if ('Gare du Nord', 'Gare de Lyon') in paths_matrix:
            paths_to_show.append(('Gare du Nord', 'Gare de Lyon'))
        if ('Gare Montparnasse', 'Gare du Nord') in paths_matrix:
            paths_to_show.append(('Gare Montparnasse', 'Gare du Nord'))
        if ('Gare de Lyon', 'Gare Saint-Lazare') in paths_matrix:
            paths_to_show.append(('Gare de Lyon', 'Gare Saint-Lazare'))
        
        if paths_to_show:
            # Créer un graphe avec les chemins
            path_nodes_set = set()
            for (start, end) in paths_to_show:
                path = paths_matrix[(start, end)]
                path_nodes_set.update(path)
                for node in path[::15]:
                    for neighbor in list(G.neighbors(node))[:2]:
                        path_nodes_set.add(neighbor)
            
            G_gares = G.subgraph(list(path_nodes_set)[:600]).copy()
            
            # Coordonnées
            pos_gares = {}
            for node in G_gares.nodes():
                if 'x' in G_gares.nodes[node] and 'y' in G_gares.nodes[node]:
                    pos_gares[node] = (G_gares.nodes[node]['x'], G_gares.nodes[node]['y'])
            
            if pos_gares:
                center_lon = sum(p[0] for p in pos_gares.values()) / len(pos_gares)
                center_lat = sum(p[1] for p in pos_gares.values()) / len(pos_gares)
                
                fig = go.Figure()
                
                # Arêtes
                edge_lons, edge_lats = [], []
                for edge in G_gares.edges():
                    if edge[0] in pos_gares and edge[1] in pos_gares:
                        lon0, lat0 = pos_gares[edge[0]]
                        lon1, lat1 = pos_gares[edge[1]]
                        edge_lons.extend([lon0, lon1, None])
                        edge_lats.extend([lat0, lat1, None])
                
                if edge_lons:
                    fig.add_trace(go.Scattermap(
                        mode='lines',
                        lon=edge_lons,
                        lat=edge_lats,
                        line=dict(width=0.3, color='lightgray'),
                        hoverinfo='none',
                        showlegend=False
                    ))
                
                # Chemins entre gares
                colors = ['blue', 'green', 'purple']
                for i, (start, end) in enumerate(paths_to_show):
                    path = paths_matrix[(start, end)]
                    path_lons = [pos_gares[node][0] for node in path if node in pos_gares]
                    path_lats = [pos_gares[node][1] for node in path if node in pos_gares]
                    
                    if path_lons and path_lats:
                        color = colors[i % len(colors)]
                        dist_info = ""
                        if (start, end) in distances_matrix:
                            dist_info = f" ({distances_matrix[(start, end)]/1000:.1f} km)"
                        fig.add_trace(go.Scattermap(
                            mode='lines+markers',
                            lon=path_lons,
                            lat=path_lats,
                            line=dict(width=4, color=color),
                            marker=dict(size=5, color=color, opacity=0.6),
                            name=f'{start} → {end}{dist_info}',
                            showlegend=True
                        ))
                
                # Marqueurs pour les gares
                gare_lons, gare_lats, gare_labels = [], [], []
                for name, node in gare_nodes.items():
                    if node in pos_gares:
                        gare_lons.append(pos_gares[node][0])
                        gare_lats.append(pos_gares[node][1])
                        gare_labels.append(name)
                
                if gare_lons:
                    fig.add_trace(go.Scattermap(
                        mode='markers',
                        lon=gare_lons,
                        lat=gare_lats,
                        text=gare_labels,
                        marker=dict(size=16, color='red'),
                        name='Gares',
                        showlegend=True
                    ))
                
                fig.update_layout(
                    title="Chemins entre Gares Parisiennes",
                    showlegend=True,
                    mapbox=dict(
                        style="open-street-map",
                        center=dict(lon=center_lon, lat=center_lat),
                        zoom=12,
                        bearing=0,
                        pitch=0
                    ),
                    margin=dict(l=0, r=0, t=40, b=0),
                    height=700
                )
                
                fig.show()
    
except Exception as e:
    print(f"Erreur lors de l'exercice 4 : {e}")
    import traceback
    traceback.print_exc()


EXERCICE 4 : Chemins entre gares parisiennes

Recherche des nœuds proches des gares...

Erreur lors de l'exercice 4 : Unknown format code 's' for object of type 'int'


Traceback (most recent call last):
  File "/var/folders/05/3bcz94ds2xlfpypg943txflc0000gn/T/ipykernel_57704/1444820083.py", line 37, in <module>
    print(f"{name:25s}: nœud {closest_node:10s} (distance: {dist_m:.0f} m)")
                             ^^^^^^^^^^^^^^^^^^
ValueError: Unknown format code 's' for object of type 'int'
